In [ ]:
%matplotlib inline
import numpy as np
import time
import scipy
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.colors as mpcol
import src.dab_util as du
import src.signal_gen as sg
import src.tcp_sync as ts
import src.dab_tuning_lib as dt
reload(sg)
reload(du)
reload(dt)

In [ ]:
import src.SendDictTcp as sdt

In [ ]:
t1 = ts.UhdSyncMsg(port=47009, packet_size=4*16384, packet_type="f"*16384)

In [ ]:
sender = sdt.SendDictTcp('127.0.0.1', 1112)

In [ ]:
sender.send({"txgain":60})

In [ ]:
sender.send({"a1":1})

In [ ]:
def measure_sholders():
    for i in range(10):
        t1.has_msg()
        msgs = t1.get_msgs(20)
        sholders = [dt.calc_signal_sholder_ratio(np.array(msg), 8000000) for msg in msgs]
        std = np.std(sholders)
        mean = np.mean(sholders)
        if std/mean < 0.05:
            continue
        else:
            return np.mean(sholders)
        raise Exception("Variance of measurement to high")

In [ ]:
measure_sholders()

In [ ]:
t1.has_msg()

In [ ]:
#top.stop()
#top.wait()